# Scaling PostgreSQL
## Database Assignment 7 (Not Mandatory)

### 1. Using a single join and a grouping, write a query that allows you to find the forename, surname, driverid and the total number of wins for each driver in the f1db schema.

In [121]:
%%sql SELECT 
    drivers.driverid as driverId,
    drivers.forename as forename,
    drivers.surname as surname,
    count(results.position) as wins
FROM drivers
JOIN results USING (driverid)
WHERE results.position = 1 
GROUP BY (drivers.driverid) 
ORDER BY wins DESC
LIMIT 5;

5 rows affected.


driverid,forename,surname,wins
30,Michael,Schumacher,91
1,Lewis,Hamilton,60
117,Alain,Prost,51
20,Sebastian,Vettel,46
102,Ayrton,Senna,41


### 2. Write a query that joins the tables results, constructors and drivers to show the amount of times a driver has driven a car from a constructor. In other words how many times one driver (for instance Schumacher) has driven a constructor (for instance Mercedes). Your table should have three fields: drivers.driverref, constructors.name and count(*) (showing the number of times the driver has driven the constructor).
Hint: You can group on more than one value

In [122]:
%%sql SELECT 
    drivers.driverref as driver,
    constructors.name as constructor,
    count(*) as time
FROM drivers
JOIN results USING (driverid)
JOIN constructors USING (constructorid)
GROUP BY (constructors.name, drivers.driverref) 
ORDER BY time DESC
LIMIT 5;

5 rows affected.


driver,constructor,time
michael_schumacher,Ferrari,181
coulthard,McLaren,150
massa,Ferrari,140
button,McLaren,137
rosberg,Mercedes,136


### 3. Now we have found the most driven vehicles even for those who didn't finish the race. Extend the query from 2 by removing all tuples from the result where the status is not "Finished".

In [41]:
%%sql
DROP VIEW IF EXISTS constructor_finished;
CREATE VIEW constructor_finished AS 
SELECT 
    drivers.driverref as driver,
    constructors.name as constructor,
    count(*) as time
FROM drivers
JOIN results USING (driverid)
JOIN constructors USING (constructorid)
WHERE results.statusid = 1
GROUP BY (constructors.name, drivers.driverref) 
ORDER BY time DESC;

Done.
Done.


[]

In [19]:
%%sql SELECT 
    drivers.driverref as driver,
    constructors.name as constructor,
    count(*) as time
FROM drivers
JOIN results USING (driverid)
JOIN constructors USING (constructorid)
WHERE results.statusid = 1
GROUP BY (constructors.name, drivers.driverref) 
ORDER BY time DESC
LIMIT 10;

10 rows affected.


driver,constructor,time
michael_schumacher,Ferrari,141
massa,Ferrari,112
rosberg,Mercedes,109
webber,Red Bull,100
vettel,Red Bull,97
raikkonen,Ferrari,92
alonso,Ferrari,87
button,McLaren,87
coulthard,McLaren,85
hamilton,Mercedes,85


### 4. Create a third (and new) query that finds the amount of milliseconds spent in pitstop (see the pitstops relation) by each unique combinations of driverid and raceid.
Hint: use the sum aggregate function to find the total amount of milliseconds
Hint: i recommend making this into a view

In [53]:
%%sql 
DROP VIEW IF EXISTS pitstop_times;
CREATE VIEW pitstop_times AS SELECT 
    drivers.driverref as driver_ref,
    drivers.driverid as driver_id,
    races.raceid as race_id,
    count(pitstops.driverid) as times,
    sum(pitstops.milliseconds) as milliseconds
FROM drivers
JOIN pitstops USING (driverid)
JOIN races USING (raceid)
GROUP BY (drivers.driverid, races.raceid);

Done.
Done.


[]

In [54]:
%sql SELECT * FROM pitstop_times ORDER BY milliseconds DESC LIMIT 5 ;

5 rows affected.


driver_ref,driver_id,race_id,times,milliseconds
ricciardo,817,967,5,3572090
button,18,967,5,3569001
max_verstappen,830,967,5,3563738
massa,13,967,4,3554774
kvyat,826,967,4,3546767


### 5. Time to put everything together. Use the query for 4 as a join subquery in query 3 (see slides on "a query within a query") to find the total amount of pitstop time, for each result in query 3

In [65]:
%%sql SELECT 
    cf.driver as driver_ref,
    cf.constructor as constructor_name,
    sum(pt.milliseconds)
FROM constructor_finished as cf
INNER JOIN (SELECT 
                pitstop_times.driver_ref as pt_driverref,
                pitstop_times.driver_id as pt_driverid,
                pitstop_times.milliseconds,
                pitstop_times.race_id, 
                results.constructorid, 
                constructors.name 
            FROM pitstop_times
            JOIN results ON results.raceid = pitstop_times.race_id
            AND results.driverid = pitstop_times.driver_id
            JOIN constructors ON constructors.constructorid = results.constructorid
           ) as pt
ON pt.pt_driverref = cf.driver AND pt.name = cf.constructor
GROUP BY (cf.driver, cf.constructor)
ORDER BY sum DESC
LIMIT 10;

10 rows affected.


driver_ref,constructor_name,sum
hamilton,Mercedes,12428578
rosberg,Mercedes,12390412
perez,Force India,11876732
massa,Williams,11874456
ricciardo,Red Bull,11816929
button,McLaren,11800713
bottas,Williams,10665109
hulkenberg,Force India,10198930
vettel,Ferrari,9248461
nasr,Sauber,8513533


### 6. As a last thing we want to find the driver with the least pistop time. However we want to control for the amount of races (see explanation below), so we need to include one last column: the average pitstop time per race. Your table should be sorted based on the ascending values of that average pitstop time.
    A. Explanation: If we just count the pitstop time, we only get the total time he spend in a workshop, no matter how many races he participated in. If a driver participated in 100 races, he would have more pitstop time than a driver participating in one race!
    B. Hint: Use the sum function to accumulate the total pistop time and divide that with the amount of races the driver has participated in

In [68]:
%sql SELECT driver_ref, AVG(milliseconds) FROM pitstop_times GROUP BY (driver_ref) ORDER BY avg ASC LIMIT 10;

10 rows affected.


driver_ref,avg
gasly,25044.000000000000
giovinazzi,29017.000000000000
glock,49466.555555555556
merhi,49585.230769230769
pic,52173.405405405405
karthikeyan,52388.291666666667
resta,52736.224137931034
michael_schumacher,53388.105263157895
rosa,54951.235294117647
petrov,55713.297297297297
